In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import pdb

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import json
import os.path as osp
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.0.0+cu118
Torchvision Version:  0.15.1+cu118


In [ ]:
for i in os.listdir("/content/drive/MyDrive/240P/train/")[:5]:  
  !tar -xf '/content/drive/MyDrive/240P/train/{i}'

In [ ]:
data_dir={}
for k in os.listdir("/content/drive/MyDrive/240P/train/")[:5]: 
  data_dir[k]=f'/content/{k[:k.index(".")]}'

In [ ]:
data_dir

{'tt0072684.tar': '/content/tt0072684',
 'tt0119738.tar': '/content/tt0119738',
 'tt0081505.tar': '/content/tt0081505',
 'tt0467406.tar': '/content/tt0467406',
 'tt1045772.tar': '/content/tt1045772'}

In [ ]:
!unzip "/content/drive/MyDrive/scene318-20230427T183512Z-001.zip"

Archive:  /content/drive/MyDrive/scene318-20230427T183512Z-001.zip
  inflating: scene318/meta/split318.json  
  inflating: scene318/label318/tt0094737.txt  
  inflating: scene318/label318/tt0159365.txt  
  inflating: scene318/label318/tt2488496.txt  
  inflating: scene318/label318/tt0443272.txt  
  inflating: scene318/label318/tt0120815.txt  
  inflating: scene318/label318/tt0091042.txt  
  inflating: scene318/label318/tt0120338.txt  
  inflating: scene318/label318/tt2024544.txt  
  inflating: scene318/label318/tt0848228.txt  
  inflating: scene318/label318/tt0120737.txt  
  inflating: scene318/label318/tt0440963.txt  
  inflating: scene318/label318/tt0499549.txt  
  inflating: scene318/label318/tt1800241.txt  
  inflating: scene318/label318/tt0780504.txt  
  inflating: scene318/label318/tt1340800.txt  
  inflating: scene318/label318/tt1375666.txt  
  inflating: scene318/label318/tt0090022.txt  
  inflating: scene318/label318/tt1100089.txt  
  inflating: scene318/label318/tt1411238.txt

In [ ]:
k

'tt0072684.tar'

In [ ]:
df_random = {}
for k in os.listdir("/content/drive/MyDrive/240P/train/")[:5]:
  with open(f'/content/scene318/label318/{k[:k.index(".")]}.txt') as f:
    lines=f.readlines()
    shot_num = []
    label = []
    for i in range(len(lines)):
      shot_num.append(lines[i].split()[0])
      label.append(lines[i].split()[1])
  
  #df_random[k]=pd.read_csv(f'/content/data/scene318/label318/{k[:k.index(".")]}.txt',sep =" ")
  #df_random[k]=df_random[k][:-1]
  data = list(zip(shot_num,label))
  df_random[k] = pd.DataFrame(data,columns=['index','label'])

In [ ]:
for k in os.listdir("/content/drive/MyDrive/240P/train/")[:5]:
  df_random[k]['index'] = f'{k[:k.index(".")]}'+'__' + df_random[k]['index'].astype(str)

In [ ]:
keys=[]
for k in os.listdir("/content/drive/MyDrive/240P/train/")[:5]:
  keys.append(k)

In [ ]:
for k in keys:
  path = []
  for i in range(df_random[k].shape[0]): 
    path.append(f"/content/{df_random[k]['index'][i].split('__')[0]}/shot_{df_random[k]['index'][i].split('__')[1]}_img_0.jpg")
  df_random[k]['path'] = path

In [ ]:
df_random[k]

,index,label,path
0,tt1045772__0000,0,/content/tt1045772/shot_0000_img_0.jpg
1,tt1045772__0001,0,/content/tt1045772/shot_0001_img_0.jpg
2,tt1045772__0002,0,/content/tt1045772/shot_0002_img_0.jpg
3,tt1045772__0003,1,/content/tt1045772/shot_0003_img_0.jpg
4,tt1045772__0004,0,/content/tt1045772/shot_0004_img_0.jpg
...,...,...,...
1031,tt1045772__1031,0,/content/tt1045772/shot_1031_img_0.jpg
1032,tt1045772__1032,1,/content/tt1045772/shot_1032_img_0.jpg
1033,tt1045772__1033,1,/content/tt1045772/shot_1033_img_0.jpg
1034,tt1045772__1034,-1,/content/tt1045772/shot_1034_img_0.jpg


In [ ]:
img_name = {}
for k in os.listdir("/content/drive/MyDrive/240P/train/")[:5]:
  img_name[k] = sorted(os.listdir(data_dir[k]))

In [ ]:
df_random[k]['index'][0].split('__')[0]

'tt1045772'

In [ ]:
from skimage.util.dtype import img_as_uint
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
from skimage import io, transform
transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
class MovieNetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        #self.df = self.df.loc[self.df.index.repeat(3)].reset_index(drop=True)
        self.root_dir = root_dir 
        # print(len(sorted(os.listdir(self.root_dir))))
        # print(len(self.df))
        self.transform = transform
        self.path =''
    '''def transform(self):
      data_transforms = {
     transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    }'''
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_st=[]
        for ids in range(0,6):
          
          #path = self.root_dir+'/'+ self.df['index'][ids+idx].split('__')[0]+'/'
          #flag = 0 
          if ids+idx<0 or ids+idx>=4600:
            continue 
          #img_name = sorted(os.listdir(path))[idx+ids]
          img_name = self.df['path'][idx+ids]
          #print(self.path+"/"+img_name)

          image = io.imread(img_name)
          #print(type(image))
          image = self.transform(image)
          image_st.append(image)
         
        #print(image)
        if len(image_st)<6:
          return {'image': [-1], 'label': [-1]}
        image=torch.stack(image_st)
        labels = np.array([self.df['label'][idx+3]])

        sample = {'image': torch.tensor(image), 'label': torch.tensor(labels)}
        return sample


In [ ]:
newdf = df_random[keys[0]].merge(df_random[keys[1]],how='outer',on=['index','label','path'])

for i in keys[2:]:
  newdf = newdf.merge(df_random[i],how='outer',on=['index','label','path'])
  print(newdf.shape)

(2490, 3)
(3587, 3)
(4623, 3)


In [ ]:
print(newdf.shape)
s = 0
for i in keys:
  s+=df_random[i].shape[0]
print(s)

(4623, 3)
4623


In [ ]:
newdf.to_csv('final_df.csv', index=False)

In [ ]:
face_dataset = MovieNetDataset(csv_file='/content/final_df.csv',
                                    root_dir='/content',transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

fig = plt.figure()

# for i in range(len(face_dataset)):
#     sample = face_dataset[i]
#     print(sample['image'].shape)
#     print(i, sample['image'].shape, sample['label'])


<Figure size 640x480 with 0 Axes>

In [ ]:
class Cos(nn.Module):
    def __init__(self):
        super(Cos, self).__init__()
        self.shot_num = 6
        self.channel = 512
        self.conv1 = nn.Conv2d(1, self.channel, kernel_size=(self.shot_num//2, 1))

    def forward(self, x):  # [batch_size, seq_len, shot_num, feat_dim]
        #print(x.shape)
        x = x.view(-1, 1, x.shape[2], x.shape[3])
        #print("cos_reshape")
        #print(x.shape)
        part1, part2 = torch.split(x, [self.shot_num//2]*2, dim=2)
        #print(part1.shape)
        # batch_size*seq_len, 1, [self.shot_num//2], feat_dim
        part1 = self.conv1(part1).squeeze()
        part2 = self.conv1(part2).squeeze()
        #print(part1.shape)
        x = F.cosine_similarity(part1, part2, dim=2)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=1)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=2)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=3)  # batch_size,channel

        return x

In [ ]:
class BNet(nn.Module):
    def __init__(self):
        super(BNet, self).__init__()
        self.shot_num = 6
        self.channel = 512
        self.conv1 = nn.Conv2d(1, self.channel, kernel_size=(6, 1))
        self.max3d = nn.MaxPool3d(kernel_size=(self.channel, 1, 1))
        self.cos = Cos()
        model_ft = models.resnet18(pretrained=True)
        model_ft.fc = nn.Identity()
        model_ft.eval()
        self.feat_extractor = model_ft

    def forward(self, x):  # [batch_size, seq_len, shot_num, 3, 224, 224]
        #print(x.shape)
        #print(type(x))
        feat = self.feat_extractor(x.float())
        feat = feat.view(feat.size(0), -1)
        feat =  feat.view(1, 10, 6, feat.shape[-1])
        
        #print("feat",feat)
        # [batch_size, seq_len, shot_num, feat_dim] 
        context=feat
        context = feat.view(
            feat.shape[0]*feat.shape[1], 1, feat.shape[-2], feat.shape[-1])
        #print("context",context)
        context = self.conv1(context)
        # batch_size*seq_len,sim_channel,1,feat_dim
        context = self.max3d(context)
        # batch_size*seq_len,1,1,feat_dim
        context = context.squeeze()
        sim = self.cos(feat)
        #print(context.shape,sim.shape)
        bound = torch.cat((context, sim), dim=1)
        return bound


In [ ]:
class LGSSone(nn.Module):
    def __init__(self, mode="image"):
        super(LGSSone, self).__init__()
        self.seq_len = 10
        self.num_layers = 1
        self.lstm_hidden_size = 512
        if mode == "image":
            self.bnet = BNet()
            self.input_dim = (512+
                512)
        self.lstm = nn.LSTM(input_size=self.input_dim,
                            hidden_size=self.lstm_hidden_size,
                            num_layers=self.num_layers,
                            batch_first=True,
                            bidirectional=True)

        self.fc1 = nn.Linear(self.lstm_hidden_size*2, 100)
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        #print(x)
        x = self.bnet(x)
        x = x.view(-1, self.seq_len, x.shape[-1])
        # torch.Size([128, seq_len, 3*channel])
        self.lstm.flatten_parameters()
        out, (_, _) = self.lstm(x, None)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        #print(out.shape)
        #out=out.flatten()
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        #out = out.view(-1, 2)
        return out

In [ ]:
class LGSS_image(nn.Module):
    def __init__(self):
        super(LGSS_image, self).__init__()
        #self.mode = cfg.dataset.mode
        #if 'image' in self.mode:
        self.bnet_image = LGSSone("image")

    def forward(self, img):
        out = 0
        image_bound = self.bnet_image(img)
        out = nn.functional.sigmoid(image_bound)
        return out

In [ ]:
model=LGSS_image()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 195MB/s]


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(5):
  c=0
  for i in range(0,10000,10):
    imgs=[]
    labels=[]
    flag=0
    for j in range(i,i+10):
      if len(face_dataset[j]['image'])==1:
        flag=1
        break
      img=(face_dataset[j]['image']) 
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    #print(flag)
    #flag=0
    if flag==1:
      #print("flag",flag)
      #print("i",i)
      #print("j",j)
      continue
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    for k in range(10):
      #print(output[k],labels[k])
      if torch.round(output[k])==labels[k]: 
        c+=1
    loss=criterion(output,labels.float())
    loss.backward()
    optimizer.step()
    #print(c)
    #print(loss.item())
    
    if i%50==0 and i>0:
      print(f"Epoch: {epoch},batch_processed: {str(i)}/10000, train_acc: {c/i}")
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, "/content/gdrive/MyDrive/checkpoint"+str(epoch)+'.pth')
  torch.save(model, '/content/gdrive/MyDrive/model'+str(epoch)+'.h5')
  

<ipython-input-10-8bea005a6677>:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'image': torch.tensor(image), 'label': torch.tensor(labels)}


Epoch: 0,batch_processed: 50/10000, train_acc: 0.56
Epoch: 0,batch_processed: 100/10000, train_acc: 0.42
Epoch: 0,batch_processed: 150/10000, train_acc: 0.32666666666666666
Epoch: 0,batch_processed: 200/10000, train_acc: 0.27
Epoch: 0,batch_processed: 250/10000, train_acc: 0.264
Epoch: 0,batch_processed: 300/10000, train_acc: 0.3466666666666667
Epoch: 0,batch_processed: 350/10000, train_acc: 0.3628571428571429
Epoch: 0,batch_processed: 400/10000, train_acc: 0.355
Epoch: 0,batch_processed: 450/10000, train_acc: 0.4066666666666667
Epoch: 0,batch_processed: 500/10000, train_acc: 0.428
Epoch: 0,batch_processed: 550/10000, train_acc: 0.4509090909090909
Epoch: 0,batch_processed: 600/10000, train_acc: 0.485
Epoch: 0,batch_processed: 650/10000, train_acc: 0.49846153846153846
Epoch: 0,batch_processed: 700/10000, train_acc: 0.5214285714285715
Epoch: 0,batch_processed: 750/10000, train_acc: 0.5413333333333333
Epoch: 0,batch_processed: 800/10000, train_acc: 0.565
Epoch: 0,batch_processed: 850/1000

In [ ]:

model.train()
for epoch in range(5):
  c=0
  for i in range(0,10000,10):
    imgs=[]
    labels=[]
    for j in range(i,i+10):
      img=(face_dataset[j]['image']) 
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    for k in range(10):
      #print(output[k],labels[k])
      if torch.round(output[k])==labels[k]: 
        c+=1
    loss=criterion(output,labels.float())
    loss.backward()
    optimizer.step()
    #print(c)
    #print(loss.item())
    
    if i%50==0 and i>0:
      print(f"Epoch: {epoch},batch_processed: {str(i)}/10000, train_acc: {c/i}")
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, "/content/gdrive/MyDrive/checkpoint"+str(epoch)+'.pth')
  torch.save(model, '/content/gdrive/MyDrive/model'+str(epoch)+'.h5')
  

In [ ]:
model.train()
for epoch in range(2):
  c=0
  for i in range(0,2000,10):
    imgs=[]
    labels=[]
    for j in range(i,i+10):
      img=(face_dataset[j]['image'])
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    for k in range(10):
      #print(output[k],labels[k])
      if torch.round(output[k])==labels[k]: 
        c+=1
    loss=criterion(output,labels.float())
    loss.backward()
    optimizer.step()
    #print(c)
    #print(loss.item())
    
    if i%50==0 and i>0:
      print(f"Epoch: {epoch},batch_processed: {i/2000}, train_acc: {c/i}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.train()
for epoch in range(1):
  c=0
  for i in range(0,1000,10):
    imgs=[]
    labels=[]
    for j in range(i,i+10):
      img=(face_dataset[j]['image'])
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    for k in range(10):
      #print(output[k],labels[k])
      if torch.round(output[k])==labels[k]: 
        c+=1
    loss=criterion(output,labels.float())
    loss.backward()
    optimizer.step()
    #print(c)
    #print(loss.item())
    
    if i%10==0 and i>0:
      print(f"Epoch: {epoch},batch_processed: {i/1000}, train_acc: {c/i}")


In [ ]:
torch.save(model, '/content/gdrive/MyDrive/model.h5')

In [ ]:
model=torch.load('/content/drive/MyDrive/model4.h5')

In [ ]:
#model=torch.load('/content/gdrive/MyDrive/model.h5')
model.eval()

LGSS_image(
  (bnet_image): LGSSone(
    (bnet): BNet(
      (conv1): Conv2d(1, 512, kernel_size=(6, 1), stride=(1, 1))
      (max3d): MaxPool3d(kernel_size=(512, 1, 1), stride=(512, 1, 1), padding=0, dilation=1, ceil_mode=False)
      (cos): Cos(
        (conv1): Conv2d(1, 512, kernel_size=(3, 1), stride=(1, 1))
      )
      (feat_extractor): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 6

In [ ]:
from sklearn.metrics import average_precision_score
def average_precision(gt,pred):
  return average_precision_score(gt, pred)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
len(face_dataset)

4623

In [ ]:
#model.eval()
c=0
outs=[]
targs=[]
for i in range(1001,1300,10):
    imgs=[]
    labels=[]
    #c=0
    flag=0
    for j in range(i,i+10):
      if len(face_dataset[j]['image'])==1:
        flag=1
        break
      img=(face_dataset[j]['image'])
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    if flag==1:
      continue
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    threshold=0.0535
    #print(output)
    output = torch.where(output < threshold, torch.zeros_like(output), torch.ones_like(output))
    for k in range(10):
      #print(output[k],labels[k])
      if torch.round(output[k])==labels[k]: 
        c+=1
    gt = torch.round(output)
    precision = average_precision(gt.detach().numpy(),labels)
    outs.append(gt.detach().numpy())
    targs.append(labels.numpy()) 
    #loss=criterion(output,labels.float())
    #loss.backward()
    #optimizer.step()
    #print(c,i)
    #print(loss.item())
    #print(f"Epoch: {epoch},batch_processed: {i-2000/350}, test_acc: {c/(i-2000)},precision: {precision}")
    if i%10==1 and i>1001:
      print(f"batch_processed: {(i-1000)}/300, test_acc: {c/(i-991)},precision: {precision}")

batch_processed: 11/300, test_acc: 0.45,precision: 0.8
batch_processed: 21/300, test_acc: 0.5333333333333333,precision: 0.3
batch_processed: 31/300, test_acc: 0.575,precision: 0.3
batch_processed: 41/300, test_acc: 0.58,precision: 0.3
batch_processed: 51/300, test_acc: 0.6166666666666667,precision: 0.5333333333333333
batch_processed: 61/300, test_acc: 0.6,precision: 0.3
batch_processed: 71/300, test_acc: 0.625,precision: 0.5444444444444444
batch_processed: 81/300, test_acc: 0.6333333333333333,precision: 0.3
batch_processed: 91/300, test_acc: 0.64,precision: 0.3
batch_processed: 101/300, test_acc: 0.6272727272727273,precision: 0.5
batch_processed: 111/300, test_acc: 0.625,precision: 0.3
batch_processed: 121/300, test_acc: 0.6076923076923076,precision: 0.3
batch_processed: 131/300, test_acc: 0.6142857142857143,precision: 0.3666666666666667
batch_processed: 141/300, test_acc: 0.62,precision: 0.3
batch_processed: 151/300, test_acc: 0.625,precision: 0.3
batch_processed: 161/300, test_acc: 0

In [ ]:
outs=np.array(outs).reshape(-1,1)
targs=np.array(targs).reshape(-1,1)

In [ ]:
AP = average_precision(outs,targs)

In [ ]:
AP

0.3470286576168929